<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/210821Word_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word 단위 RNN
https://wikidocs.net/64765  로 다른 문장으로 직접 따라 실습한다.

# 훈련 데이터의 전처리

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = 'the quick brown fox jumps over the lazy dog'.split()
vocab = list(set(sentence))
print(vocab)

['the', 'quick', 'fox', 'jumps', 'dog', 'brown', 'lazy', 'over']


vocab을 단어장으로 활용할려고 한다.
이제 단어장의 단어 들에 고유한 정수 인덱스를 부여한다.

In [8]:
word2index = {tkn : i for i, tkn in enumerate(vocab, 1)}
# 단어장에 없는 단어를 의미하는 토큰을 추가한다.
word2index['<unk>'] = 0
print(word2index)

# 인덱스를 다시 단어로 바꿔주는 딕셔너리도 만들어준다.
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{'the': 1, 'quick': 2, 'fox': 3, 'jumps': 4, 'dog': 5, 'brown': 6, 'lazy': 7, 'over': 8, '<unk>': 0}
{1: 'the', 2: 'quick', 3: 'fox', 4: 'jumps', 5: 'dog', 6: 'brown', 7: 'lazy', 8: 'over', 0: '<unk>'}


In [9]:
# 훈련 데이터 설계
# 마지막 단어를 제외하고 문장을 비우고, 문장을 완성하는 문제를 풀 것이다.

def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [11]:
X, Y = build_data(sentence, word2index)

## 모델의 구현
RNN을 활용하여 모델을 만들고자 한다.

In [13]:
class Net(nn.Module):
  def __init__(self, vocab_size, input_size, hidden_size, batch_first = True):
    super(Net, self).__init__()
    self.embedding_layer = nn.Embedding(num_embeddings= vocab_size,
                                        embedding_dim = input_size)
    self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first)
    self.linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
    output = self.embedding_layer(x)
    output, hidden = self.rnn_layer(output)
    output = self.linear(output)

    return output.view(-1, output.size(2))

In [14]:
# hyperparameter
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [15]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

output = model(X)
print(output)

tensor([[-0.0524,  0.1287,  0.1051, -0.0219,  0.2135,  0.0424, -0.0127, -0.2107,
          0.1589],
        [-0.1561, -0.3218,  0.0215,  0.0784,  0.1254,  0.1570,  0.0123, -0.0559,
          0.1838],
        [-0.2528,  0.3364,  0.2524, -0.1313,  0.2534, -0.2731, -0.0207, -0.3205,
          0.1208],
        [-0.1287, -0.2458,  0.0111,  0.0878,  0.2104, -0.0155,  0.0402, -0.1065,
          0.1548],
        [-0.0290,  0.3623,  0.2548,  0.1784,  0.5064, -0.2112, -0.2846, -0.1630,
         -0.2226],
        [-0.0101, -0.1975,  0.1773,  0.0819,  0.1948,  0.0265,  0.1447, -0.0145,
          0.0460],
        [-0.0361,  0.1567,  0.1094,  0.0575,  0.3404, -0.1210, -0.1082, -0.2475,
          0.2264],
        [-0.1662, -0.0385,  0.1395, -0.1015,  0.0748, -0.0856, -0.1084, -0.0587,
          0.0455]], grad_fn=<ViewBackward>)


In [16]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [17]:
# 훈련 시작
for step in range(201):
    optimizer.zero_grad()

    output = model(X)
    loss = loss_function(output, Y.view(-1))

    loss.backward()
    optimizer.step()
    
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.2998 
Repeat jumps over the jumps jumps jumps jumps quick

[41/201] 1.8815 
Repeat quick brown fox jumps jumps brown fox dog

[81/201] 1.2948 
Repeat quick brown fox jumps over the lazy dog

[121/201] 0.7270 
Repeat quick brown fox jumps over the lazy dog

[161/201] 0.3990 
Repeat quick brown fox jumps over the lazy dog

[201/201] 0.2309 
Repeat quick brown fox jumps over the lazy dog

